# EVENTS TABLE

This notebook collects event data from the StubHub API for the Delaware North vanues listed below.  

In [1]:
# Base modules
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
import time
import re

# API modules
from getpass import getpass
import base64 #To encode app key and app secret
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Venue names and IDs

StubHub IDs were collected from URLs on venue pages. 

In [2]:
# {location_name : stubhub_id}

locations = {
    'Times Union Center' : '4585',
    'PPG Paints Arena' : '147209', 
}

# Turn venue IDs into list
ids = list(filter(None, list(locations.values())))

# Invert above dictionary to replace values in dataframe below
locations_ = {value: key for key, value in locations.items()}

## STUBHUB API

## Token

StubHub generates an access token for developers from its token API.  The code below generates a token each time a request is sent to the token API.

**The username and password must be for a StubHub *user account* - not the developer account.  This login info cannot be the same as the developer account.**  The consumer key and secret are available when the developer registers an app.

In [3]:
USERNAME = '####################'
PASSWORD = getpass('PASSWORD: ')
CONSUMER_KEY = '####################'
CONSUMER_SECRET = '####################'

combo = CONSUMER_KEY + ':' + CONSUMER_SECRET

# Encode key & secret for passing into header
# Convert bytes to string
auth_token = base64.b64encode(combo.encode('utf-8'))
auth_token = str(auth_token.decode())

# Request access token from Stubhub Token API
TOKEN_API = 'https://api.stubhub.com/sellers/oauth/accesstoken?grant_type=client_credentials'
REFRESH_API = 'https://api.stubhub.com/sellers/oauth/refresh_accesstoken?grant_type=refresh_token'

header = {
    'Content-Type' : 'application/json',
    'Authorization' : 'Basic ' + auth_token
}

body = {
    'username' : USERNAME,
    'password' : PASSWORD,
    'scope' : 'PRODUCTION'
}

try:
    r = requests.post(TOKEN_API, headers = header, json = body).json()
    token = r['access_token']
    print('Success - token acquired.')
    print('Token: %s' %(token))
except:
    print('ERROR, status code %s' %(r.status_code), 'Response text: '%(r.text))

PASSWORD: ········
Success - token acquired.
Token: 0HFDykRGjSIs3gzZfq56fzVGqq5C


## Request StubHub data

StubHub will not allow multiple venue IDs in one query.  The code below sends a request for each venue.

Free use is limited to 500 requests per day, and rows are limited to 500 max.

This code collects event data within a specified range from today's date.

In [4]:
# Define date range (days) since today to collect event data for
RANGE = 365

d1 = datetime.datetime.today().date()
d2 = d1 + timedelta(days = RANGE)

# Format date range for request body
dates = str(d1) + ' TO ' + str(d2)

In [5]:
ROWS = 400

url = 'https://api.stubhub.com/sellers/search/events/v3?'

events = {}

header = {
    'Content-Type' : 'application/json',
    'Authorization' : 'Bearer ' + token
}

# Make API request for each venue to get list of events
for venue in ids:
    
    try:

        body = {
            'venueId' : venue,
            'dateLocal' : dates,
            'sort' : 'eventDateLocal asc', # Must use sort to get closest upcoming events
            'rows' : ROWS
        }

        events_r = requests.get(url, headers = header, params=body).json()
        events[venue] = events_r
        
    except:
        print('ERROR, status code %s' %(events_r.status_code), 'Response text: '%(events_r.text))
        

# Create list of dicts for each event
data = []

for venue in events.keys():
    for event in events[venue]['events']:
        data.append(event)

#### Example of event object

In [6]:
data[0]

{'id': 104252557,
 'status': 'Active',
 'locale': 'en_US',
 'name': 'Holy Cross Crusaders at Siena Saints Basketball',
 'description': 'Buy and sell Holy Cross Crusaders vs Siena Saints Basketball Tickets for Dec 29 at Times Union Center in Albany at StubHub! Tickets are 100% guaranteed by FanProtect.',
 'webURI': 'siena-saints-basketball-tickets-siena-saints-basketball-albany-times-union-center-12-29-2019/event/104252557/',
 'eventDateLocal': '2019-12-29T14:00:00-0500',
 'eventDateUTC': '2019-12-29T19:00:00+0000',
 'createdDate': '2019-07-26T21:59:39+0000',
 'lastUpdatedDate': '2019-10-18T18:10:41+0000',
 'hideEventDate': False,
 'hideEventTime': False,
 'venue': {'id': 4585,
  'name': 'Times Union Center',
  'city': 'Albany',
  'state': 'NY',
  'postalCode': '12207',
  'country': 'US',
  'venueConfigId': 350639,
  'venueConfigName': 'Basketball - Dynamic'},
 'timezone': 'EST',
 'performers': [{'id': 377744, 'name': 'Siena Saints Basketball'},
  {'id': 173494, 'name': 'Holy Cross Crus

## Create DataFrame

In [7]:
# Define cols - add 'dow', 'holiday', 'givaway' when dataframe is created
cols = ['location_id', 'event_id', 'date', 'start_time', 'category_1', 'category_2', 
        'category_3', 'performer_1', 'performer_2', 'event_name', 'local', 'utc', 
        'venue_id', 'venue_name', 'city', 'state', 'zip_code']

#Initialize list of list to convert to df
df_ = []

# Create list of lists (rows) containing data that corresponds to above cols
for event in data:
    li = []
    
    # Venue ID - will be replaced with DN location_id
    li.append(str(event['venue']['id']))
    
    # Event ID - Stubhub's event identifier
    li.append(str(event['id']))
    
    # Date - Date and Time separated by 'T'
    t = event['eventDateLocal'].split('T', 1)
    li.append(t[0])
    
    # Start time
    li.append(t[1][:5])
     
    # Category 1 - i.e. 'Sports' or 'Concert'
    li.append(event['ancestors']['categories'][0]['name'])
    
    # Category 2 - i.e. 'MLB' or 'NFL'
    # This is not useful data for concerts
    if li[4] != 'Concert':
        try:
            li.append(event['ancestors']['groupings'][0]['name'])
        except:
            li.append('')
    else:
        li.append('')
    
    # Category 3 - i.e. 'Regular' or 'playoffs'
    try:
        li.append(event['ancestors']['groupings'][1]['name'])
    except:
        li.append('')
        
    # Performer 1 - i.e. Home team or musician
    try:
        li.append(event['performers'][0]['name'])
    except:
        li.append('')
    
    # Performer 2 - i.e. Opponent or opening act
    try:
        li.append(event['performers'][1]['name'])
    except:
        li.append('')
    
    # Name
    li.append(event['name'])
    
    # Local timestamp (string)
    li.append(event['eventDateLocal'])
    
    # UTC timestamp (string)
    li.append(event['eventDateUTC'])
    
    # Venue ID - keep this field in table
    li.append(str(event['venue']['id']))
    
    # Venue name
    li.append(event['venue']['name'])
    
    # Venue city
    li.append(event['venue']['city'])
    
    # Venue state
    li.append(event['venue']['state'])
    
    # Venue zip code
    li.append(event['venue']['postalCode'])
    
    # Add event to list
    df_.append(li) 

# Create dataframe and convert date
events_df = pd.DataFrame(df_, columns=cols)
events_df['date'] = pd.to_datetime(events_df['date'])

# Convert local and UTC strings to timestamp
events_df['local'] = pd.to_datetime(events_df['local'])
events_df['utc'] = pd.to_datetime(events_df['utc'])

# Convert start time to date time
events_df['start_time'] = pd.to_datetime(events_df['start_time'], format='%H:%M').dt.time

# Create column for DOW
events_df['dow'] = events_df['date'].dt.day_name()

# Replace values with location name
events_df['location_id'].replace(locations_, inplace=True)

# Alter nml location_ids to match DN database (Giants and Jets)
nml_teams = ['giants', 'jets']
for team in nml_teams:
    events_df['location_id'] = np.where(
        events_df['performer_1'].str.contains(team, flags=re.IGNORECASE),
            'nml:' + team, events_df['location_id'])

# Flag holidays
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
holidays = calendar().holidays(start=events_df['date'].min(), end=events_df['date'].max())
events_df['holiday'] = events_df['date'].isin(holidays).astype(int)

# Flag givaways (for sports)
events_df['giveaway'] = events_df['event_name'].str.contains('Giveaway', flags=re.IGNORECASE).astype(int)

# Sort by date
events_df.sort_values(by=['date','location_id','start_time'], ascending=True, inplace=True)

events_df.head(n=20)

,location_id,event_id,date,start_time,category_1,category_2,category_3,performer_1,performer_2,event_name,local,utc,venue_id,venue_name,city,state,zip_code,dow,holiday,giveaway
34,PPG Paints Arena,104215119,2019-12-28,19:00:00,Sports,NHL,NHL Regular Season,Pittsburgh Penguins,Nashville Predators,Nashville Predators at Pittsburgh Penguins,2019-12-28 19:00:00-05:00,2019-12-29 00:00:00+00:00,147209,PPG Paints Arena,Pittsburgh,PA,15219,Saturday,0,0
0,Times Union Center,104252557,2019-12-29,14:00:00,Sports,NCAA Basketball,MAAC Basketball,Siena Saints Basketball,Holy Cross Crusaders Basketball,Holy Cross Crusaders at Siena Saints Basketball,2019-12-29 14:00:00-05:00,2019-12-29 19:00:00+00:00,4585,Times Union Center,Albany,NY,12207,Sunday,0,0
35,PPG Paints Arena,104215120,2019-12-30,19:00:00,Sports,NHL,NHL Regular Season,Pittsburgh Penguins,Ottawa Senators,Ottawa Senators at Pittsburgh Penguins Tickets...,2019-12-30 19:00:00-05:00,2019-12-31 00:00:00+00:00,147209,PPG Paints Arena,Pittsburgh,PA,15219,Monday,0,1
36,PPG Paints Arena,104375039,2019-12-31,15:00:00,Theater and Arts,,,Jeff Dunham,,Jeff Dunham,2019-12-31 15:00:00-05:00,2019-12-31 20:00:00+00:00,147209,PPG Paints Arena,Pittsburgh,PA,15219,Tuesday,0,0
37,PPG Paints Arena,104215508,2020-01-02,19:00:00,Sports,NHL,NHL Regular Season,Pittsburgh Penguins,San Jose Sharks,San Jose Sharks at Pittsburgh Penguins,2020-01-02 19:00:00-05:00,2020-01-03 00:00:00+00:00,147209,PPG Paints Arena,Pittsburgh,PA,15219,Thursday,0,0
1,Times Union Center,104318969,2020-01-03,19:00:00,Sports,NCAA Basketball,MAAC Basketball,Siena Saints Basketball,Monmouth Hawks Basketball,Monmouth Hawks at Siena Saints Basketball,2020-01-03 19:00:00-05:00,2020-01-04 00:00:00+00:00,4585,Times Union Center,Albany,NY,12207,Friday,0,0
38,PPG Paints Arena,104215511,2020-01-05,17:00:00,Sports,NHL,NHL Regular Season,Pittsburgh Penguins,Florida Panthers,Florida Panthers at Pittsburgh Penguins Ticket...,2020-01-05 17:00:00-05:00,2020-01-05 22:00:00+00:00,147209,PPG Paints Arena,Pittsburgh,PA,15219,Sunday,0,1
2,Times Union Center,104319199,2020-01-09,19:00:00,Sports,NCAA Basketball,MAAC Basketball,Siena Saints Basketball,Saint Peter's Peacocks Basketball,Saint Peter's Peacocks at Siena Saints Basketball,2020-01-09 19:00:00-05:00,2020-01-10 00:00:00+00:00,4585,Times Union Center,Albany,NY,12207,Thursday,0,0
39,PPG Paints Arena,104403368,2020-01-10,20:00:00,Theater and Arts,,,Jim Gaffigan,,Jim Gaffigan,2020-01-10 20:00:00-05:00,2020-01-11 01:00:00+00:00,147209,PPG Paints Arena,Pittsburgh,PA,15219,Friday,0,0
40,PPG Paints Arena,104225158,2020-01-11,19:05:00,Sports,NCAA Hockey,Atlantic Hockey,Robert Morris Colonials Hockey,Penn State Nittany Lions Hockey,Penn State Nittany Lions at Robert Morris Colo...,2020-01-11 19:05:00-05:00,2020-01-12 00:05:00+00:00,147209,PPG Paints Arena,Pittsburgh,PA,15219,Saturday,0,0


## SPOTIFY - CONCERT GENRES TABLE

The Stubhub API does not provide genres for artists.  Genre could be a significant predictor for transactions volume (i.e. country concerts will sell more alcohol).  Artists from the concerts in the table can be assigned genres with the Spotify API, which is accessed through the Spotipy module.

Spotipy requires installation of the python requests package.  User must register app with Spotify Developer platform and provide client ID & secret.

This code also collects the image URL for each artist for use in a potential dashboard.

In [8]:
# Enter Spotify credentials
CLIENT_ID = '####################'
CLIENT_SECRET = '####################'

# Subset concert events and retrieve unique list of artists
concerts = events_df[ events_df['category_1'] == 'Concert' ]
artists = list(set(concerts['performer_1']))

# Initialize genres list for tuples
genres = []

# Initialize images dict of urls for future dashboard
images = {}

# Authenticate
client_credentials = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials)

# Collect data
for artist in artists:
    try:
        # Create tuples with artist name and genres
        a = spotify.search(q='artist:' + artist, type='artist')
        b = a['artists']['items'][0]['genres']
        
        # Some artists do not have values for the genre key
        if len(b) > 0: 
            for genre in b:
                c = (artist, genre)
                genres.append(c)
        else:
            c = (artist, '')
            genres.append(c)
                
        # Add image URL
        images[artist] = a['artists']['items'][0]['images'][0]['url']
    # Some artists do not have a genre key
    except:
        continue

In [9]:
# Create genres df for joining with events table
genres_df = pd.DataFrame(genres, columns=['performer_1', 'genre'])

genres_df.head(n=10)

,performer_1,genre
0,Celine Dion,canadian pop
1,Celine Dion,dance pop
2,Ringo Starr,album rock
3,Ringo Starr,bubblegum pop
4,Ringo Starr,classic rock
5,Ringo Starr,folk rock
6,Ringo Starr,mellow gold
7,Ringo Starr,rock
8,Ringo Starr,singer-songwriter
9,Ringo Starr,soft rock


## UPDATE TABLES

This code merges the data collected with the existing tables (pickle files) and deletes duplicate events.

In [10]:
# Load previous data
events_table = pd.read_pickle('events_table.pkl')
events_table.reset_index(inplace=True)
genres_table = pd.read_pickle('genres_table.pkl')
genres_table.reset_index(inplace=True)

# Concat new events data
events_ = pd.concat([events_table, events_df], axis=0, sort=True)

# Drop duplicate events based on unique event_id.  Keep last row in case data was updated (new start time, etc.)
events_.drop_duplicates(subset='event_id', keep='last', inplace=True)

# Sort events data
events_.sort_values(by=['date','location_id','start_time'], inplace=True)
events_ = events_.reindex(columns=['location_id', 'event_id', 'dow', 'date', 'start_time', 'category_1',
            'category_2', 'category_3', 'performer_1', 'performer_2', 'event_name', 'local', 'utc', 'holiday', 
            'giveaway', 'venue_id', 'venue_name', 'city', 'state', 'zip_code'])
events_.set_index('location_id', inplace=True)

# Concat genres
genres_ = pd.concat([genres_table, genres_df], axis=0, sort=True)

# Drop duplicate artist-genre pairs
genres_.drop_duplicates(subset=['performer_1','genre'], inplace=True)

# Sort genres
genres_.sort_values(by=['performer_1'], inplace=True)
genres_.set_index('performer_1', inplace=True)

# Upload new tables
events_.to_pickle('events_table.pkl')
genres_.to_pickle('genres_table.pkl')